<a href="https://colab.research.google.com/github/astuteIsrael/3D-Rotate-CSS/blob/main/fifth_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [73]:
# Load the datasets
variable_definitions = pd.read_csv('/content/Variable_Definitions.csv')
train_data = pd.read_csv('/content/Train Dataset .csv')
test_data = pd.read_csv('/content/Test Dataset.csv')
sample_submission = pd.read_csv('/content/Sample Submission.csv')

# Display first few rows to inspect the data
print(train_data.head())
print(test_data.head())


      Id  Age  Sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0  16167   33    0   1       158   205    1        0      154      0      1.5   
1  11275   53    1   2       198   154    0        1      104      0      0.8   
2  13251   37    1   2       101   202    1        0      155      0      2.1   
3  19921   75    0   0       113   306    1        2       88      1      4.9   
4  11293   35    1   2       139   419    1        1      166      1      0.9   

   slope  ca  thal  target  
0      1   4     1       1  
1      2   1     0       0  
2      1   3     1       1  
3      0   2     2       1  
4      2   4     0       1  
      id  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0  16501   70    1   0       163   495    0        2      170      1      2.0   
1  10444   61    1   0       131   238    0        2       74      1      4.9   
2  14288   53    1   0        95   558    1        1       73      1      0.7   
3  10409   37  

In [80]:
# Check for missing values and handle them
print(train_data.isnull().sum())

# Fill missing values (example with median, can be customized)
train_data.fillna(train_data.median(), inplace=True)

# Split features (X) and target (y)
X = train_data.drop(columns=['target'])
y = train_data['target']

# Scale the features (Standardization)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Id          0
Age         0
Sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [81]:
# Split data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [82]:
# Initialize XGBoost classifier
xgb_model = XGBClassifier(random_state=42)

# Set up Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation on XGBoost model
xgb_cv_results = cross_val_score(xgb_model, X_scaled, y, cv=kfold, scoring='accuracy', verbose=2)
print(f"XGBoost Cross-Validation Accuracy: {xgb_cv_results.mean()} (+/- {xgb_cv_results.std()})")

[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.3s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
XGBoost Cross-Validation Accuracy: 0.811721845611469 (+/- 0.007004245831598185)


In [83]:
# Initialize Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Create an ensemble model using VotingClassifier
ensemble_model = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rf_model)], voting='soft')

# Perform cross-validation on Ensemble model
ensemble_cv_results = cross_val_score(ensemble_model, X_scaled, y, cv=kfold, scoring='accuracy', verbose=2)
print(f"Ensemble Model Cross-Validation Accuracy: {ensemble_cv_results.mean()} (+/- {ensemble_cv_results.std()})")

[CV] END .................................................... total time=   1.8s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s
Ensemble Model Cross-Validation Accuracy: 0.8104895314712197 (+/- 0.006030021430305819)


In [84]:
# Train the ensemble model on the full training set
ensemble_model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = ensemble_model.predict(X_val)

# Evaluate the ensemble model on validation data
accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred)

print(f"Validation Accuracy (Ensemble Model): {accuracy}")
print(f"Validation AUC-ROC (Ensemble Model): {roc_auc}")
print("Classification Report (Ensemble Model):")
print(classification_report(y_val, y_val_pred))

Validation Accuracy (Ensemble Model): 0.8288843258042436
Validation AUC-ROC (Ensemble Model): 0.7045619035419202
Classification Report (Ensemble Model):
              precision    recall  f1-score   support

           0       0.52      0.51      0.52       260
           1       0.89      0.90      0.90      1201

    accuracy                           0.83      1461
   macro avg       0.71      0.70      0.71      1461
weighted avg       0.83      0.83      0.83      1461



In [85]:
# Scale the test data using the same scaler
test_data_scaled = scaler.transform(test_data)

# Predict on the test dataset using the ensemble model
test_predictions = ensemble_model.predict(test_data_scaled)

# Prepare the submission file
sample_submission['target'] = test_predictions
sample_submission.to_csv('Ensemble_Heart_Disease_Predictions.csv', index=False)

print("Submission file 'Ensemble_Heart_Disease_Predictions.csv' generated successfully.")

Submission file 'Ensemble_Heart_Disease_Predictions.csv' generated successfully.
